In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from sklearn.metrics import mean_squared_error

In [16]:
# Load the log dataset (replace with your dataset file)
data = pd.read_csv('../dataset/log_data.csv')

In [17]:
# Convert timestamp to datetime objects
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [18]:
# Calculate time difference in seconds from the minimum timestamp
min_timestamp = data['timestamp'].min()
data['time_diff'] = (data['timestamp'] - min_timestamp).dt.total_seconds()

In [19]:
# Select relevant features for anomaly detection
# In this case, we'll use time_diff, activity, and username
features = data[['time_diff', 'activity', 'username']]

In [20]:
# Convert categorical features to numerical using label encoding
label_encoders = {}
for col in ['activity', 'username']:
    label_encoders[col] = LabelEncoder()
    features[col] = label_encoders[col].fit_transform(features[col])

C:\Users\arunh\AppData\Local\Temp\ipykernel_9636\1446478807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = label_encoders[col].fit_transform(features[col])
C:\Users\arunh\AppData\Local\Temp\ipykernel_9636\1446478807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = label_encoders[col].fit_transform(features[col])


In [21]:
# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [22]:
# Split the data into train and test sets
X_train, X_test = train_test_split(scaled_features, test_size=0.2, random_state=42)


In [23]:
# Build the autoencoder model
input_dim = X_train.shape[1]
encoding_dim = 10  # Adjust based on your data complexity
autoencoder = Sequential()
autoencoder.add(Dense(encoding_dim, input_shape=(input_dim,), activation='relu'))
autoencoder.add(Dense(input_dim, activation='linear'))
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [24]:
# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

Epoch 1/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4057 - val_loss: 0.1530
Epoch 2/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0924 - val_loss: 0.0556
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0389 - val_loss: 0.0266
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0194 - val_loss: 0.0145
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0113 - val_loss: 0.0093
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0077 - val_loss: 0.0067
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0057 - val_loss: 0.0050
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0045 - val_loss: 0.0039
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0036 - val_loss: 0.0031
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0028 - val_loss: 0.0025

In [25]:
autoencoder.save('autoencoder_model.h5')

In [26]:
# Reconstruct the data using the trained autoencoder
reconstructed_data = autoencoder.predict(scaled_features)
mse = np.mean(np.power(scaled_features - reconstructed_data, 2), axis=1)


313/313 [==============================] - 0s 806us/step


In [27]:
# Define a threshold to identify anomalies
threshold = np.percentile(mse, 95)  # Adjust based on your data

In [28]:
# Identify anomalies
anomalies = data[mse > threshold]
print("Anomalies:")
print(anomalies)

Anomalies:
               timestamp      activity          username        source_ip  \
20   2023-01-25 07:35:07   ProductView         zsheppard   166.16.220.249   
22   2023-06-04 11:32:37   ProductView          alexis64  112.217.112.210   
57   2023-03-05 07:52:45  AccessDenied     underwoodjohn   205.87.244.162   
73   2023-02-01 21:51:53  AccessDenied       thomasowens    11.97.211.168   
96   2023-01-21 03:08:19  AccessDenied         stacyholt     50.188.52.59   
...                  ...           ...               ...              ...   
9811 2023-03-13 18:58:59  AccessDenied          yhammond    191.215.22.40   
9888 2023-02-08 07:41:35  AccessDenied       walshshelby   145.189.39.201   
9940 2023-01-18 18:52:18  AccessDenied        raysanders   92.103.120.131   
9974 2023-01-25 13:57:07  AccessDenied  richardrodriguez    64.166.90.107   
9976 2023-01-30 09:12:18   OrderPlaced           wendy17   199.27.255.238   

       action   result   time_diff  
20     Failed  Success   21